In [1]:
!pip install yolov5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt


fatal: destination path 'yolov5' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Include all packages
import os
import cv2
from time import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from yolov5.models.yolo import Model
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# import zipfile
# with zipfile.ZipFile('/content/drive/MyDrive/DL/FinalProject/signDatabasePublicFramesOnly.zip', 'r') as zip_ref:
#     zip_ref.extractall('./DataSet')


In [5]:

def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    # cv2.circle(image, (x1, y1), radius=5, color=(0, 0, 255), thickness=-1)
    # cv2.circle(image, (x2, y2), radius=5, color=(0, 0, 255), thickness=-1)
    # cv2.circle(resizedImage, (x1New, y1New), radius=5, color=(0, 0, 255), thickness=-1)
    # cv2.circle(resizedImage, (x2New, y2New), radius=5, color=(0, 0, 255), thickness=-1)
    # cv2.imshow('Resized Image', resizedImage)
    # cv2.imshow('Original Image', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return resizedImage, x1New, y1New, x2New, y2New


In [6]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [uniqueSigns.index(row[1]), row[2], row[3], row[4], row[5]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


In [7]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        [label, x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
        resizedImages.append(resizedImage)
        newAnnotations.append(
            [(i % batchSize), label, x1New, y1New, x2New, y2New])

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label


In [9]:
def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [10]:
def TargetstoTensors(targets, batch_size, num_anchors, grid_sizes, numClasses):
    target_obj = []
    target_cls = []
    target_box = []
    # batch_size = 32
    for grid_size in grid_sizes:
        target_obj.append(torch.zeros(
            (batch_size, num_anchors, grid_size, grid_size, 1)))
        target_cls.append(torch.zeros(
            (batch_size, num_anchors, grid_size, grid_size, numClasses)))
        target_box.append(torch.zeros(
            (batch_size, num_anchors, grid_size, grid_size, 4)))
    # current_batch_size = int(targets[:, 0].max()) + 1
    # print("current_batch_size",targets.size(0))
    for target in targets:
        batch_index, cls, x_center, y_center, width, height = target.long()

        for i, grid_size in enumerate(grid_sizes):

            x_cell, y_cell = int(
                x_center * grid_size), int(y_center * grid_size)
            anchor = 0
            try:
                target_obj[i][batch_index, anchor, y_cell, x_cell, 0] = 1
                target_cls[i][batch_index, anchor, y_cell, x_cell, cls] = 1
                target_box[i][batch_index, anchor, y_cell, x_cell] = torch.tensor(
                    [x_center, y_center, width, height])
            except Exception as e:
                # print(e)
                pass
    return target_obj, target_cls, target_box


In [11]:
class YOLOv5Loss(nn.Module):
    def __init__(self, numClasses, num_anchors=3):
        super(YOLOv5Loss, self).__init__()
        self.numClasses = numClasses
        self.num_anchors = num_anchors

    def forward(self, preds, targets):
        obj_loss = torch.tensor(0.0, device=preds[0].device)
        cls_loss = torch.tensor(0.0, device=preds[0].device)
        box_loss = torch.tensor(0.0, device=preds[0].device)
        batch_size = preds[0].size(0)
        grid_sizes = [pred.size(2) for pred in preds]
        target_obj_list, target_cls_list, target_box_list = TargetstoTensors(
            targets, batch_size, self.num_anchors, grid_sizes, self.numClasses)
        for i, pred in enumerate(preds):
            # pred_obj = pred[..., 4].sigmoid()
            # pred_cls = pred[..., 5:].sigmoid()
            # pred_box = pred[..., :4]

            target_obj = target_obj_list[i].to(pred.device)
            target_cls = target_cls_list[i].to(pred.device)
            target_box = target_box_list[i].to(pred.device)

            obj_loss += nn.BCEWithLogitsLoss()(pred[..., 4:5], target_obj)
            cls_loss += nn.BCEWithLogitsLoss()(pred[..., 5:], target_cls)
            box_loss += nn.MSELoss()(pred[..., :4], target_box)

        total_loss = obj_loss + cls_loss + box_loss
        return total_loss


In [12]:
def CreateYolov5Model(numClasses: int, version="s"):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    return model


In [13]:
def TrainModel(model, dataLoader, epochs, optimizer, lossFunction, device):
    model.train()
    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch+1, epochs))
        startTime = time()
        totalLoss = 0
        dataLoaderLen = len(dataLoader)
        for i, (inputs, targets) in enumerate(dataLoader):
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = lossFunction(outputs, targets)
                loss.backward()
                optimizer.step()

            totalLoss += loss.item() * inputs.size(0)

        endTime = time()
        timeTaken = endTime-startTime
        if(((i*100)//dataLoaderLen) % 10 == 0):
            print((i*100//dataLoaderLen), end="%,")
        epochLoss = totalLoss / dataLoaderLen
        print()
        print("Training Loss: {:.4f}".format(epochLoss))
        print("Time taken: {}min, {}, secs".format(timeTaken//60, timeTaken % 60))

    print("Training complete.")
    return model


In [14]:
batchSize = 32
inputShape = (416, 416)
epochs = 100
num_anchors = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
print("Using {} device".format(device))

Using cuda device


In [16]:
images, annotations, numClasses = LoadDataSet("./DataSet")


In [17]:
X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations


In [18]:
trainDataLoader, valDataLoader = CreateDataLoaders(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val


In [19]:
yolov5_model = CreateYolov5Model(numClasses)
optimizer = optim.Adam(yolov5_model.parameters(), lr=0.001)
yolov5LossFunction= YOLOv5Loss(numClasses=numClasses)
yolov5_model = yolov5_model.to(device)
yolov5LossFunction = yolov5LossFunction.to(device)

Overriding model.yaml nc=80 with nc=47

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]            

In [20]:
trainedModel = TrainModel(yolov5_model, trainDataLoader, epochs, optimizer, yolov5LossFunction, device)

Epoch 1/100:

Training Loss: 0.8658
Time taken: 1.0min, 40.77568006515503, secs
Epoch 2/100:

Training Loss: 0.0319
Time taken: 1.0min, 31.014641046524048, secs
Epoch 3/100:

Training Loss: 0.0123
Time taken: 1.0min, 30.041335344314575, secs
Epoch 4/100:

Training Loss: 0.0069
Time taken: 1.0min, 29.933436632156372, secs
Epoch 5/100:

Training Loss: 0.0049
Time taken: 1.0min, 30.31379723548889, secs
Epoch 6/100:

Training Loss: 0.0043
Time taken: 1.0min, 30.519484043121338, secs
Epoch 7/100:

Training Loss: 0.0028
Time taken: 1.0min, 29.938671827316284, secs
Epoch 8/100:

Training Loss: 0.0032
Time taken: 1.0min, 30.096277713775635, secs
Epoch 9/100:

Training Loss: 0.0022
Time taken: 1.0min, 29.949742317199707, secs
Epoch 10/100:

Training Loss: 0.0023
Time taken: 1.0min, 29.825109720230103, secs
Epoch 11/100:

Training Loss: 0.0023
Time taken: 1.0min, 29.956392288208008, secs
Epoch 12/100:

Training Loss: 0.0029
Time taken: 1.0min, 29.86660075187683, secs
Epoch 13/100:

Training Loss

In [22]:
torch.save(yolov5_model.state_dict(), 'trained_yolov5_modelv3.pth')